# Day 1: The setup with kaggle mnist dataset

In [1]:
import kaggle
import keras
import torch
%load_ext tensorboard
# check for gpu
!nvidia-smi
# check for cuda
!nvcc --version
# use gpu
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

2024-05-03 22:30:12.627514: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-03 22:30:13.544082: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Fri May  3 22:30:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070 ...    Off | 00000000:01:00.0 Off |                  N/A |
| N/A   36C    P8               9W / 115W |      8MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

2024-05-03 22:30:16.022667: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
2024-05-03 22:30:16.022698: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] retrieving CUDA diagnostic information for host: ab2c2a999337
2024-05-03 22:30:16.022706: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:141] hostname: ab2c2a999337
2024-05-03 22:30:16.022857: I external/local_xla/xla/stream_executor/cuda/cuda_d

Purpose of this notebook is to:
* run kaggle mnist locally
* on gpu
* and upload the results to kaggle.

##  Download the dataset from kaggle

In [2]:
import os
import zipfile

# Define the directory where you want to download the data
data_dir = "MyDataset/mnist/raw"  # './' represents the current directory

# Check if the directory exists
if not os.path.exists(data_dir):
    # If not, create the directory
    os.makedirs(data_dir)

# Move to that directory
os.chdir(data_dir)
competition_name = "digit-recognizer"
# Download the data
os.system("kaggle competitions download -c " + competition_name)

# Unzip the data
with zipfile.ZipFile("digit-recognizer.zip","r") as zip_ref:
    zip_ref.extractall(".")

os.chdir("../../..")

100%|██████████| 15.3M/15.3M [00:03<00:00, 4.63MB/s]


## Load Data And prepare the data

In [4]:
import pandas as pd
import numpy as np

# Load the dataset
train_df = pd.read_csv('MyDataset/mnist/raw/train.csv')
test_df = pd.read_csv('MyDataset/mnist/raw/test.csv')

# Split features and labels
y_train = train_df["label"]
x_train = train_df.drop(labels = ["label"], axis = 1)

# Convert to numpy arrays
x_train = x_train.values
y_train = y_train.values

# Test data
x_test = test_df.values



### Inspet the data

In [5]:
train_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Prepare the data

In [6]:
# Make Validation set
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.1, random_state=2)

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)
x_val = x_val.reshape(-1, 28, 28, 1)
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

x_train shape: (37800, 28, 28, 1)
y_train shape: (37800,)
37800 train samples
28000 test samples


## Make model

In [7]:
num_classes = 10
input_shape = (28, 28, 1)

model = keras.Sequential(
    [
        keras.layers.Input(shape=input_shape),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation="softmax"),
    ]
)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 24, 24, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 10, 10, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 8, 8, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 260,298 (1016.79 KB)

 Trainable params: 260,298 (1016.79 KB)

 Non-trainable params: 0 (0.00 B)

### Compile the model

In [8]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name="acc"),
    ],
)

### Callbacks   

In [9]:
callbacks = [ 
    keras.callbacks.EarlyStopping(monitor="val_loss",patience=10),
    keras.callbacks.ModelCheckpoint("mnist_model.keras", save_best_only=True),
    keras.callbacks.TensorBoard(log_dir="./logs"),
]
%tensorboard --logdir=logs

### Fit the model

In [10]:
batch_size = 128
epochs = 20

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=(x_val, y_val),

)

score = model.evaluate(x_val, y_val, verbose=0)

Epoch 1/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 53s 176ms/step - acc: 0.4925 - loss: 1.4080 - val_acc: 0.9567 - val_loss: 17.3352
Epoch 2/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 51s 174ms/step - acc: 0.9252 - loss: 0.2538 - val_acc: 0.9590 - val_loss: 23.6405
Epoch 3/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 51s 174ms/step - acc: 0.9504 - loss: 0.1711 - val_acc: 0.9783 - val_loss: 11.4378
Epoch 4/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 51s 173ms/step - acc: 0.9609 - loss: 0.1300 - val_acc: 0.9731 - val_loss: 12.3887
Epoch 5/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 51s 173ms/step - acc: 0.9668 - loss: 0.1090 - val_acc: 0.9802 - val_loss: 10.5507
Epoch 6/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 51s 173ms/step - acc: 0.9713 - loss: 0.0974 - val_acc: 0.9800 - val_loss: 9.8040
Epoch 7/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 51s 173ms/step - acc: 0.9746 - loss: 0.0861 - val_acc: 0.9864 - val_loss: 6.1548
Epoch 8/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 52s 174ms/step - acc: 0.9782 - loss: 0.0751 - val_acc: 0.9879 - val_loss: 6.5883
Epoch 9/20
296/296 ━━━━━━━━

In [11]:

import pandas as pd
# Assuming 'predictions' is an array containing your model's predictions
predictions = model.predict(x_test)


875/875 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step


In [12]:
class_predictions = np.argmax(predictions, axis=1)
# Create a DataFrame with the prediction results
# 'ImageId' is a common column name in MNIST-like competitions
submission = pd.DataFrame({
    "ImageId": list(range(1, len(class_predictions) + 1)),
    "Label": class_predictions
})

# Save the DataFrame to a CSV file
submission.to_csv('my_submission.csv', index=False)

In [18]:
#!kaggle competitions submit -c digit-recognizer -f my_submission.csv -m "First submission"


fileName           date                 description       status    publicScore  privateScore  
-----------------  -------------------  ----------------  --------  -----------  ------------  
my_submission.csv  2024-05-04 08:02:09  First submission  complete  0.9926                     
100%|█████████████████████████████████████████| 208k/208k [00:00<00:00, 253kB/s]
Successfully submitted to Digit Recognizer

In [ ]:
!kaggle competitions submissions digit-recognizer